<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Remove-all-rows-with-missing-values-in-order-delivered_carrier_date-or-order_delivered_customer_date" data-toc-modified-id="Remove-all-rows-with-missing-values-in-order-delivered_carrier_date-or-order_delivered_customer_date-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Remove all rows with missing values in order delivered_carrier_date or order_delivered_customer_date</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm

sns.set()

# Remove Outliers and Time Format

In [2]:
import pandas as pd

def clean_data(df=None):
    if df is not None:
        df = time_format(df)
        df = create_order_session(create_order_weekday(df))
        df = remove_empty_rows(create_metric_dimension_product(df))
        df = remove_geolocation_outliers(df)
        return df

def create_order_weekday(df=None):
    if df is None:
        return
    df['order_purchase_weekday'] = df['order_purchase_timestamp'].dt.day_name()
    df = df.assign(order_purchase_session=pd.cut(df.order_purchase_timestamp.dt.hour, [0, 6, 12, 18, 24],
                                                 labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False))
    return df

def create_order_session(df=None):
    if df is None:
        return
    winter = [12, 1, 2]
    spring = [3, 4, 5]
    summer = [6, 7, 8]
    autumn = [9, 10, 11]
    d = {**dict.fromkeys(winter, 'winter'), **dict.fromkeys(spring, 'spring'), **dict.fromkeys(summer, 'summer'),
         **dict.fromkeys(autumn, 'autumn')}

    df['season'] = list(map(d.get, df.order_purchase_timestamp.dt.month))
    return df

def create_metric_dimension_product(df=None):
    if df is not None:
        df['product_weight_kg'], df['product_weight_g'] = df['product_weight_g'] // 1000, \
                                                          df['product_weight_g'] % 1000
        df['product_length_m'], df['product_length_cm'] = df['product_length_cm'] // 100, \
                                                          df['product_length_cm'] % 100
        df['product_height_m'], df['product_height_cm'] = df['product_height_cm'] // 100, \
                                                          df['product_height_cm'] % 100
        df['product_width_m'], df['product_width_cm'] = df['product_width_cm'] // 100, \
                                                        df['product_width_cm'] % 100
    return df

def remove_empty_rows(df=None):
    if df is None:
        return
    df = df[df['product_category_name_english'].notna()]
    return df


def time_format(data):
    '''
    Function to convert dataset time columns into pandas datetime and calculations for promise date,
    approval time and total time to deliver
    '''
    data.order_purchase_timestamp = pd.to_datetime(data.order_purchase_timestamp, format='%d/%m/%Y %H:%M')
    data.order_approved_at = pd.to_datetime(data.order_approved_at, errors='coerce', format='%d/%m/%Y %H:%M')
    data.order_estimated_delivery_date = pd.to_datetime(data.order_estimated_delivery_date, errors='coerce', format='%d/%m/%Y %H:%M')
    data.order_delivered_customer_date = pd.to_datetime(data.order_delivered_customer_date, errors='coerce', format='%d/%m/%Y %H:%M')
    df.order_delivered_carrier_date = pd.to_datetime(df.order_delivered_carrier_date, errors='coerce', format='%d/%m/%Y %H:%M')
    data['promise_date'] = data.order_estimated_delivery_date >= data.order_delivered_customer_date  # True if product delivered before or at estimated delivery date
    data['approval_time'] = data.order_approved_at - data.order_purchase_timestamp  # Time for buyer to approve sale
    #data['total_time_to_deliver'] = data.order_delivered_customer_date - data.order_purchase_timestamp  # total time from pruchase to delivery
    #data['actual_delivery_time'] = data.order_delivered_customer_date - data.order_delivered_carrier_date  # total time from pruchase to delivery
    data['actual_delivery_time'] = ((pd.to_datetime(df['order_delivered_customer_date']) - 
                            pd.to_datetime(df['order_delivered_carrier_date']))
                                .dt.total_seconds() / (60 * 60))
    data['total_delivery_time'] = ((pd.to_datetime(df['order_delivered_customer_date']) - 
                            pd.to_datetime(df['order_purchase_timestamp']))
                                .dt.total_seconds() / (60 * 60))

    return data

def remove_geolocation_outliers(data):
    # Removing some outliers 
    #Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
    data = data[data.geolocation_lat <= 5.27438888]
    #it's most Western spot is at 73 deg, 58′ 58.19″W Long.
    data = data[data.geolocation_lng >= -73.98283055]
    #It's most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
    data = data[data.geolocation_lat >= -33.75116944]
    #It's most Eastern spot is 34 deg, 47′ 35.33″ W Long.
    data = data[data.geolocation_lng <=  -34.79314722]
    return data

def null_dates(data):
    '''
    Function to find indexes of order's with null times, most of these rows are order_delievered_customer_date.
    This is probably due to cancelations
    '''
    na = data[['order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date',
               'order_delivered_customer_date']][data[
        ['order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date',
         'order_delivered_customer_date']].isna().any(1)].index

    return na

df = pd.read_csv('../../data/interim/consolidated_ecommerce_olist_1_sample.csv' , dtype={'seller_zip_code_prefix': str, 'customer_zip_code_prefix': str})
df = clean_data(df)

In [3]:
def create_order_hour_and_date(df=None):
    df['order_purchase_hour'] = df.order_purchase_timestamp.dt.hour
    df['order_purchase_date'] = df.order_purchase_timestamp.dt.date
    df['product_volumetric_weight'] = (df.product_weight_kg + df.product_weight_g )* (df.product_height_m + df.product_height_cm) * (df.product_width_m + df.product_width_cm)
    return df

df = create_order_hour_and_date(df)

### Remove all rows with missing values in order delivered_carrier_date or order_delivered_customer_date

In [4]:
df = df[(df['order_delivered_carrier_date'].isnull() == False) & (df['order_delivered_customer_date'].isnull() == False)]

In [5]:
df_geo = pd.read_csv('../../data/processed/processed_olist_geolocation_dataset.csv', dtype={'geolocation_zip_code_prefix': str})

In [6]:
df_geo.geolocation_zip_code_prefix = df_geo.geolocation_zip_code_prefix.apply(lambda x: '0' + x if(len(x) == 4) else x)
df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].apply(lambda x: '0' + x if(len(x) == 4) else x)
df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].apply(lambda x: '0' + x if(len(x) == 4) else x)


# Skyway Distance

In [7]:
from math import sin, cos, sqrt, atan2, radians

def compute_distance(row):
    # Source: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
    # approximate radius of earth in km
    R = 6373.0
    customer = row.customer_zip_code_prefix
    seller = row.seller_zip_code_prefix
    seller_location = df_geo[df_geo.geolocation_zip_code_prefix == seller]
    customer_location = df_geo[df_geo.geolocation_zip_code_prefix == customer]
    if (seller_location.shape[0] == 0) or (customer_location.shape[0] == 0):
        return None

    lat_seller = radians(seller_location.geolocation_lat.iloc[0])
    lng_seller = radians(seller_location.geolocation_lng.iloc[0])
    
    lat_customer = radians(customer_location.geolocation_lat.iloc[0])
    lng_customer = radians(customer_location.geolocation_lng.iloc[0])
    dlon = lng_seller - lng_customer
    dlat = lat_seller - lat_customer

    a = sin(dlat / 2)**2 + cos(lat_seller) * cos(lat_customer) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [8]:
df['distance'] = df[['customer_zip_code_prefix', 'seller_zip_code_prefix']].apply(compute_distance, axis=1)

In [9]:
def create_order_hour_and_date(df=None):
    df['order_purchase_hour'] = df.order_purchase_timestamp.dt.hour
    df['order_purchase_date'] = df.order_purchase_timestamp.dt.date
    df['product_area'] = ((df.product_height_m * 100) + df.product_height_cm) * ((df.product_width_m * 100) + df.product_width_cm)
    df['product_volume'] = ((df.product_height_m * 100) + df.product_height_cm) * ((df.product_width_m * 100) + df.product_width_cm) * ((df.product_length_m * 100) + df.product_length_cm)
    return df

df = create_order_hour_and_date(df)

In [10]:
df = df.drop(['Unnamed: 0'], axis=1)

#### Remove all rows with missing values in order delivered_carrier_date or order_delivered_customer_date
df = df[(df['order_delivered_carrier_date'].isnull() == False) & (df['order_delivered_customer_date'].isnull() == False)]

## Product Category Transformation


In [11]:
idx = df['order_id'].value_counts()[df['order_id'].value_counts() > 1].index
# Data contains orders with multiple items
df_multiple_product_cat_order = df[df.order_id.isin(idx)]
# Data contains orders with single item
df_single_product_cat_order = df[~df.order_id.isin(idx)]

# Filter all orders with multiple items but only have single product category
only_1_product_cat = []
for id in idx:
    if len(df_multiple_product_cat_order[df_multiple_product_cat_order.order_id == id].product_category_name_english.unique()) == 1:
        only_1_product_cat.append(id)
        
print('# orders having more than 1 items: ', len(df_multiple_product_cat_order))
print('# orders having more than 1 items BUT only have 1 product category: ', len(only_1_product_cat))

df_single_product_cat_multiple_item_order = df_multiple_product_cat_order[df_multiple_product_cat_order.order_id.isin(only_1_product_cat)]
df_single_product_cat = df_single_product_cat_order.append(df_single_product_cat_multiple_item_order)

df_grp_product_cat = df_single_product_cat.groupby('product_category_name_english').mean().reset_index().sort_values(by='total_delivery_time', ascending=True)

# Encode product categories into numerical values
df_grp_product_cat['encode_product_cat'] = np.linspace(1, df_grp_product_cat.shape[0], df_grp_product_cat.shape[0])
df_grp_product_cat = df_grp_product_cat[['product_category_name_english', 'encode_product_cat']]
df_grp_product_cat = df_grp_product_cat.set_index('product_category_name_english')
df_grp_product_cat = df_grp_product_cat.to_dict()


df_multiple_product_cat = df_multiple_product_cat_order[~df_multiple_product_cat_order.isin(df_single_product_cat_multiple_item_order).all(1)]

imputed_product_cat_for_order = {}
for id in df_multiple_product_cat.order_id.unique():
    df_order = df_multiple_product_cat[df_multiple_product_cat.order_id == id]
    unique_product_cat = df_order.product_category_name_english.unique()
    max_product_cat = unique_product_cat[0]
    for i in range(1, len(unique_product_cat)):
        if df_grp_product_cat['encode_product_cat'][max_product_cat] < df_grp_product_cat['encode_product_cat'][unique_product_cat[i]]:
            max_product_cat = unique_product_cat[i]
    imputed_product_cat_for_order[df_order.order_id.iloc[0]] = max_product_cat
    
df['product_category_name_english'] = df['order_id'].map(lambda x: imputed_product_cat_for_order[x] if x in imputed_product_cat_for_order \
                                                         else df[df.order_id == x].product_category_name_english.iloc[0])


idx = df['order_id'].value_counts()[df['order_id'].value_counts() > 1].index

double_check_df = df[df['order_id'].isin(idx)]

# Filter all orders with multiple items but only have single product category
only_1_product_cat = []
for id in idx:
    if len(double_check_df[double_check_df.order_id == id].product_category_name_english.unique()) > 1:
        only_1_product_cat.append(id)

max_column = ['distance', 'order_item_id', 'review_id']
mean_column = ['review_score']
sum_column = ['freight_value', 'product_weight_kg', 'product_weight_g', 'product_width_cm', \
              'product_width_m', 'product_length_m', 'product_length_cm', \
              'product_height_m', 'product_height_cm', \
              'price', 'payment_value', 'payment_installments', \
              'product_volume', 'product_area']

agg_dict = {}
for col in df.columns:
    if col in max_column:
        agg_dict[col] = 'max'
    elif col in mean_column:
        agg_dict[col] = 'mean'
    elif col in sum_column:
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'unique'
        
df = df.groupby('order_id', sort=False).agg(agg_dict)

for col in df.columns:
    if (col not in max_column) or (col not in mean_column) or (col not in sum_column):
        if isinstance(df[col].iloc[0], np.ndarray):
            df[col] = df[col].apply(lambda x: x[0])
            
df.order_purchase_session = df.order_purchase_session.apply(lambda x: x[0]) 

# orders having more than 1 items:  375
# orders having more than 1 items BUT only have 1 product category:  173


# Update Season For Brazil

In [12]:
def create_season(df=None):
    if df is None:
        return
    df['order_purchase_timestamp'] =  pd.to_datetime(df['order_purchase_timestamp'], format='%Y/%m/%d %H:%M:%S')
    dry = [9, 10, 11, 12, 1, 2]
    wet = [3, 4, 5, 6, 7, 8]
    d = {**dict.fromkeys(dry, 'dry'), **dict.fromkeys(wet, 'wet')}
    df['seasons'] = list(map(d.get, df.order_purchase_timestamp.dt.month))
    return df

df = create_season(df)

df.seasons = df.seasons.astype('category')
df.payment_type = df.payment_type.astype('category')

# Remove all orders with invalid delivery time
df = df[df.total_delivery_time > df.actual_delivery_time]
df = df[df.actual_delivery_time > 0]

# Connection between Cities & States

In [13]:
def calculate_time_difference_in_hour(data, col1, col2, target_col):
    data[target_col] = ((pd.to_datetime(data[col1]) - 
                            pd.to_datetime(data[col2]))
                                .dt.total_seconds() / (60 * 60))
    return

def encode_categorical_variable_by_delivery_time(data, target_col_name, asc=True, rev=False):
    df_cat_to_val = data.groupby(target_col_name).mean().reset_index().sort_values(by='total_delivery_time', ascending=asc)
    # Encode categorical val into numerical val
    encode_col_name = 'encode_' + target_col_name
    if rev:
        df_cat_to_val[encode_col_name] = np.linspace(1, df_cat_to_val.shape[0], df_cat_to_val.shape[0])
    else:
        df_cat_to_val[encode_col_name] = np.linspace(df_cat_to_val.shape[0], 1, df_cat_to_val.shape[0])
    df_cat_to_val = df_cat_to_val[[target_col_name, encode_col_name]]
    df_cat_to_val = df_cat_to_val.set_index(target_col_name)
    df_cat_to_val = df_cat_to_val.to_dict()
    # Map encoded vals into dataset
    new_target_col_name = target_col_name + '_encoded'
    data[new_target_col_name] = data[target_col_name].map(lambda x: df_cat_to_val[encode_col_name][x])
    
    return


df['total_delivery_time_log'] = np.log(df['total_delivery_time'])
df['actual_delivery_time_log'] = np.log(df['actual_delivery_time'])
df['product_weight'] = (df['product_weight_kg'] * 1000) + df['product_weight_g']
calculate_time_difference_in_hour(df, 'shipping_limit_date', 'order_purchase_timestamp', 'lag_time')
calculate_time_difference_in_hour(df, 'order_delivered_carrier_date', 'order_purchase_timestamp', 'lag_time')
encode_categorical_variable_by_delivery_time(df, 'product_category_name_english', asc=True, rev=False)

df_states = df.groupby(['customer_state', 'seller_state']).mean()
df_states = df_states['total_delivery_time'].reset_index()

df_state_connection_grp = df.groupby(['customer_state', 'seller_state']).mean().sort_values(by='total_delivery_time', ascending=True).reset_index()

# Encode pair of customer state & seller state into numeric value
df_state_connection_grp['connection_between_states'] = np.linspace(df_state_connection_grp.shape[0], 1, df_state_connection_grp.shape[0])
df_state_connection_grp = df_state_connection_grp[['customer_state', 'seller_state', 'connection_between_states']]
df_state_connection_grp = df_state_connection_grp.set_index(['customer_state', 'seller_state'])
df_state_connection_grp = df_state_connection_grp.to_dict()

# Map each pair of customer state & seller state to the corresponding numerical value in dataset
df['connection_between_states'] = df.apply(lambda x: df_state_connection_grp['connection_between_states'][(x.customer_state, x.seller_state)], axis=1)


df_city_connection_grp = df.groupby(['customer_city', 'seller_city']).mean().sort_values(by=['total_delivery_time'], ascending=True).reset_index()

# Encode pair of customer state & seller state into numeric value
df_city_connection_grp['connection_between_cities'] = np.linspace(df_city_connection_grp.shape[0], 1, df_city_connection_grp.shape[0])
df_city_connection_grp = df_city_connection_grp[['customer_city', 'seller_city', 'connection_between_cities']]
df_city_connection_grp = df_city_connection_grp.set_index(['customer_city', 'seller_city'])
df_city_connection_grp = df_city_connection_grp.to_dict()

# Map each pair of customer state & seller state to the corresponding numerical value in dataset
df['connection_between_cities'] = df.apply(lambda x: df_city_connection_grp['connection_between_cities'][(x.customer_city, x.seller_city)], axis=1)


In [14]:
df.set_index('order_id', inplace=True)

rdpc_df = pd.read_csv('../features/rdpc_features.csv')
df = pd.merge(df, rdpc_df, on='order_id')

In [15]:
df['zip_2_prefix_cat'] = df['zip_2_prefix'] = df.zip_2_prefix.astype('category').cat.codes

df.rename(columns={'review_score_x': 'review_score', 'distance_x': 'distance', 'seasons_x': 'seasons', 'lag_time_x': 'lag_time', 
                  'connection_between_states_x': 'connection_between_states','connection_between_cities_x': 'connection_between_cities',
                  'total_delivery_time_x': 'total_delivery_time'}, inplace=True)

del df['total_delivery_time_y']

df['connection_between_states'] = df['connection_between_states'].astype('int64')

df['connection_between_cities'] = df['connection_between_cities'].astype('int64')

# Seller Historical Mean

In [16]:
np.seterr(divide = 'ignore')


# df = df.sort_values(by=['order_purchase_timestamp'])
seller_delivery = df.groupby(['customer_city','seller_city','seller_id','order_purchase_timestamp'],as_index=False)[['total_delivery_time']].agg(['count','sum'])
seller_delivery.columns = [' '.join(col) for col in seller_delivery.columns]
seller_delivery.reset_index(inplace=True)

seller_delivery['total_delivery_time_CumSum'] = seller_delivery.groupby(['customer_city','seller_city','seller_id'])[['total_delivery_time sum']].cumsum()
seller_delivery['total_delivery_time_CumCount'] = seller_delivery.groupby(['customer_city','seller_city','seller_id'])[['total_delivery_time count']].cumsum()
seller_delivery['total_delivery_time_CumMean'] = seller_delivery['total_delivery_time_CumSum'] / seller_delivery['total_delivery_time_CumCount']
seller_delivery['seller_delivery_PrevMean'] = seller_delivery.groupby(['customer_city','seller_city','seller_id'])['total_delivery_time_CumMean'].shift(0)

df = df.merge(seller_delivery[['customer_city','seller_city','seller_id','order_purchase_timestamp','seller_delivery_PrevMean']],how='left',on=['customer_city','seller_city','seller_id','order_purchase_timestamp'])

def FillNone(x):
    if np.isnan(x['seller_delivery_PrevMean']):
        return x['total_delivery_time']
    else:
        return x['seller_delivery_PrevMean']
    
df['seller_delivery_PrevMean']=df.apply(lambda x : FillNone(x),axis=1)

In [17]:
df.to_csv('../../data/processed/20th_May_olist.csv')